In [ ]:
# data_preprocessing.ipynb

import os
import mne
import numpy as np
import matplotlib.pyplot as plt
from mne.preprocessing import ICA
from sklearn.preprocessing import StandardScaler


数据预处理的详细步骤和可视化
这个笔记本主要执行数据加载、去噪和标准化处理，此外还包括数据可视化部分。

python
复制代码


In [ ]:
# 1. 加载数据
def load_raw_data(raw_data_path):
    """
    加载原始EEG数据
    支持两种格式：.edf (MNE库处理) 和 .mat (SciPy加载)
    
    :param raw_data_path: 原始数据所在路径
    :return: EEG信号数据和相关元数据
    """
    data = []
    for filename in os.listdir(raw_data_path):
        filepath = os.path.join(raw_data_path, filename)
        if filepath.endswith('.edf'):
            # 使用MNE库加载edf文件
            raw = mne.io.read_raw_edf(filepath, preload=True)
            data.append(raw)
        elif filepath.endswith('.mat'):
            # 使用SciPy加载mat文件
            mat = sio.loadmat(filepath)
            data.append(mat)
    return data

In [ ]:
# 2. 预处理数据（去噪、标准化）
def preprocess_data(raw_data):
    """
    对EEG数据进行去噪和标准化处理
    
    :param raw_data: 原始EEG数据
    :return: 预处理后的EEG数据
    """
    # 去伪影：应用ICA去眼动伪影
    ica = ICA(n_components=20, random_state=97, max_iter=800)
    ica.fit(raw_data)
    raw_data_clean = ica.apply(raw_data)
    
    # 标准化：将EEG数据进行z-score标准化
    raw_data_clean = raw_data_clean.get_data()
    scaler = StandardScaler()
    raw_data_clean = scaler.fit_transform(raw_data_clean.reshape(-1, raw_data_clean.shape[-1]))
    
    return raw_data_clean

In [ ]:
# 3. 可视化原始数据与去伪影后的数据
def plot_raw_data(raw_data, title='Raw EEG Data'):
    """
    可视化EEG信号（原始数据）
    
    :param raw_data: EEG数据
    :param title: 标题
    """
    plt.figure(figsize=(10, 6))
    raw_data.plot(title=title, show=True)


In [ ]:
# 4. 示例：加载和预处理数据
raw_data_path = 'path_to_raw_data/'  # 原始数据的路径
raw_data = load_raw_data(raw_data_path)

# 可视化原始数据
plot_raw_data(raw_data[0], title="Original EEG Data")

# 预处理数据
preprocessed_data = preprocess_data(raw_data[0])

# 可视化去伪影后的数据
plt.figure(figsize=(10, 6))
plt.plot(preprocessed_data.T)
plt.title('Preprocessed EEG Data')
plt.show()